In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import pandas as pd

In [3]:
from src.shared_code import connectors as conn, dataset, utils
from src import data_assets, project_env

In [4]:
project_env.set_env_vars()

In [5]:
folder = "testing_io"
ext = "csv"

file_in = "test_reading"
file_out = "test_writting"

In [6]:
path_in = utils.path_data_raw / folder / f"{file_in}.{ext}"
path_out = utils.path_data_interim / folder/ f"{file_out}.{ext}"

# Prerequisites

In [7]:
path_folder = utils.path_data_raw / folder

if not (path_folder).is_dir():
    print(f"Creating folder: {path_folder}")
    path_folder.mkdir(parents=True)
    

In [8]:
data = pd.DataFrame([[1,2,3], [4,5,6]], columns=["x1", "x2", "x3"])

In [9]:
print(f"{path_in.as_posix()}")

C:/Users/MRomero/Documents/repos/labs/repo_template/data/raw/testing_io/test_reading.csv


In [10]:
data.to_csv( path_in, index=False)

# Local Connection

In [11]:
conn.connectionsEnum.LOCAL

<connectionsEnum.LOCAL: 'local'>

In [12]:
client_local = conn.connection_client_factory(
    connection_kind=conn.connectionsEnum.LOCAL,
    connection_name="conn_local")

conn_local = conn.DataConnectorLocal(
    connector_client=client_local,
    layer="raw",
    path=f"{folder}/{file_in}.{ext}"
)

dataset_local = dataset.DatasetPandas(conn_local)
df_train_local = dataset_local.read(func=pd.read_csv)

df_train_local.head()

,x1,x2,x3
0,1,2,3
1,4,5,6


In [13]:
dataset_local.metadata()

{'features': [],
 'target': [],
 'id_cols': [],
 'split_col': [],
 'features_num': [],
 'features_cat': []}

In [14]:
import sys

sys.getsizeof(dataset_local)

48

In [21]:
df_train_local.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   x1      2 non-null      int64
 1   x2      2 non-null      int64
 2   x3      2 non-null      int64
dtypes: int64(3)
memory usage: 176.0 bytes


# Data Connections Factory

## Define Sources

In [22]:
source_local = data_assets.sources[0]
source_local

DataAsset(name='testing_source', kind='local', layer='raw', path='testing_io/test_reading', extension='csv', description='features and labels from UCI datasets')

## Define Sinks

In [23]:
sink_local = data_assets.sinks[0]
sink_local

DataAsset(name='testing_sink', kind='local', layer='raw', path='testing_io/test_writting', extension='csv', description='features and labels from UCI datasets')

## Define Data Connections

In [24]:
conn_fac_local_source = conn.data_connections_factory(source_local)
conn_fac_local_sink = conn.data_connections_factory(sink_local)

## Read Data

In [18]:
ds_src_raw_local = dataset.DatasetPandas(conn_fac_local_source)
df_in_local = ds_src_raw_local.read(func=pd.read_csv)

df_in_local.head()

,x1,x2,x3
0,1,2,3
1,4,5,6


## Write Data

In [19]:
ds_sink_raw_local = dataset.DatasetPandas(conn_fac_local_sink)

In [20]:
ds_sink_raw_local.write(
    df_train_local, kwargs={"index_label": False})